## Predict curve in the specific time 
    - using RNN model

### Import libraries

In [49]:
import torch
import torchvision
import torch.nn as nn                        # Activation Funciton , Algo
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.utils.data as Data              # Data loader 
import numpy as np

### Sample data

    - [b , time_step , input_size] -> [b , 50 , 1]

In [61]:
num_timestep = 50                                                   # 預測多長時間的數據 if 50 , 0-49 , 然後預測 1-50 , 每個時間點有50個數據
input_size = 1
hidden_size = 10

start = np.random.randint(3 , size=1 )[0]                           # random pick number from 0-2
time_step = np.linspace(start , start + 10 ,num_timestep )          # time_step 固定頭尾 if start = 1 , end = 11
data = np.sin(time_step)
data = data.reshape(num_timestep , 1)                               # data 維度 -> [50 , 1]
x = torch.tensor(data[:-1]).float().view(1 , num_timestep -1 , 1)   # x 維度 -> [1 , 49 , 1]
y = torch.tensor(data[1:]).float().view(1 , num_timestep -1 , 1)


### Build network

In [56]:
class RNN(nn.Module):
    def __init__(self , ):
        super(RNN , self).__init__()

        self.rnn = nn.RNN(
            input_size = input_size,
            hidden_size = hidden_size,
            num_layers = 1 ,           # 多少層的 hidden layer
            batch_first = True   
        )

        self.out = nn.Linear(hidden_size , 1)

    # hidden_prev -> h0 傳進來的 hidden 狀態
    def forward(self ,x , hidden_prev):
        # out dim -> [b , time_step , hidden_size]
        # h   dim -> [b , num_layer , hidden_size]
        out , hidden_prev = self.rnn(x , hidden_prev)
        # [1 , seq , h] -> [seq , h]
        out = out.view(-1 , hidden_size)
        # [seq , h] -> [seq , 1]
        out = self.out(out)
        # 插入一個維度 維度要跟 y 相同  [1 , seq , 1] => [batch , seq , out]
        out = out.unsqueeze(dim=0)

        return out , hidden_prev


        


### Train

In [59]:
model = RNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters() , lr=0.1)


# h0 => [ batch , num_layer , hidden_size]
hidden_prev = torch.zeros(1,1,hidden_size)

for iter in range(100):
    start = np.random.randint(10, size=1 )[0]                           # random pick number from 0-2
    time_step = np.linspace(start , start + 10 ,num_timestep )          # time_step 固定頭尾 if start = 1 , end = 11
    data = np.sin(time_step)
    data = data.reshape(num_timestep , 1)                               # data 維度 -> [50 , 1]
    x = torch.tensor(data[:-1]).float().view(1 , num_timestep -1 , 1)   # x 維度 -> [1 , 49 , 1]
    y = torch.tensor(data[1:]).float().view(1 , num_timestep -1 , 1)


    output , hidden_prev = model(x , hidden_prev)
    hidden_prev = hidden_prev.detach()

    loss = criterion(output , y)
    model.zero_grad()
    loss.backward()
    optimizer.step()

RNN(
  (rnn): RNN(1, 10, batch_first=True)
  (out): Linear(in_features=10, out_features=1, bias=True)
)